In [155]:
import pandas as pd
import numpy as np
from datetime import datetime

In [156]:
# read excel with country names and codes
# return list with country codes relevant for UCDP
country_code_pathfile = '/Users/sabine.a.joseph/Documents/Country_Feature_documentation/Country_codes_NAMO.xlsx'
country_code_column_name = 'UCDP_country_codes'
sheet = 'Sheet1'

def country_codes_from_excel(country_codes, sheet_num, column_name):
    countries = pd.ExcelFile(country_codes)
    countries = countries.parse(sheet_num)
    return list(countries[column_name]) # UCDP uses Gleditsch and Ward country codes

CCS = country_codes_from_excel(country_code_pathfile, sheet, country_code_column_name)

In [157]:
UCDP_filename = '/Users/sabine.a.joseph/Downloads/ged50-csv/ged50.csv'
country_code_column_name = 'gwno'

def csv_to_df(filename, country_codes, csv_country_code_column):
    df = pd.read_csv(filename , sep=',')
    return df[df[csv_country_code_column].isin(country_codes)]

df = csv_to_df(UCDP_filename, CCS, country_code_column_name)

In [158]:
# delete not to be used columns
# 'id' # only keep for data cleaning 
vars_to_del = ['relid', 'year', 'active_year', 'conflict_dset_id', 'conflict_name', 'dyad_dset_id',
               'dyad_new_id', 'dyad_name', 'side_a_dset_id', 'side_a_new_id', 'side_a', 'side_b_dset_id',
               'side_b_new_id', 'side_b', 'source_date', 'source_headline', 'source_office', 'source_original',
               'where_prec', 'where_coordinates', 'where_description', 'adm_1', 'adm_2', 'latitude', 'longitude',
               'geom_wkt', 'priogrid_gid', 'country', 'region', 'isocc', 'gwab']

def del_columns_from_df(col_names):
    for i in col_names:
        del df[i]
    return df

df = del_columns_from_df(vars_to_del)

In [154]:
# str index to datetime
df = df.reset_index(drop=True)

df_datestring_column_name = 'date_start'
dateformat = '%Y-%m-%d'

def str_to_datetime(col_name, dateformat):
    return [datetime.strptime(str(df[col_name][i]), dateformat) 
            for i in range(0, len(df[col_name])) if i is not None]

df[df_datestring_column_name] = str_to_datetime(df_datestring_column_name, dateformat)


(63482, 17)


In [ ]:
# filter by type of crisis --- check with Oliver
# type_of_violence # 1: state-based conflict # 2: non-state conflict # 3: one-sided violence
df = df[df['type_of_violence'].isin([1, 3])]


# df = df[df.date_prec > 1] 

# ? conflict_new_id #as filter? -- existing conflict -- ask Oliver
#df.loc[(df['conflict_new_id'] == 426) & df['type_of_violence'].isin([1, 3]))]
#df.loc[(df['type_of_violence'].isin([426, 234]) & df['type_of_violence'].isin([1, 3]))]

# filter by event_clarity?

### to use:
# date var: 
# combination of: date_start, date_end
# date_prec #duration of event, only chose date_prec = 1 (verify with Oliver?)
# df = df.loc[(df['date_prec'] == 1)]

# -> date_start = date_end - chose as event date
# or always chose date_start as date
# del df.date_end

# number_of_sources #importance - only valid for yr 2013 and 2014
# del df.event_clarity


In [170]:
# grouping by date per country code
col_name_date = 'date_start'
col_name_country_codes = 'gwno'
agg_col_names = ['deaths_a', 'deaths_b', 'deaths_civilians', 'deaths_unknown', 'best_est', 'high_est', 'low_est']
event_count_col_name = 'count_num_daily_events'

def group_by_country_code_date_agg_sum(date, CC, col_name_list, ct_col_name):
# sum of death counts
# count of events per day per country code
    df[ct_col_name] = 1 
    col_name_list.append(ct_col_name)
    return df.groupby([date, CC]).agg(dict.fromkeys(col_name_list, sum))
    
df_agg = group_by_country_code_date_agg_sum(col_name_date, col_name_country_codes, agg_col_names, event_count_col_name)
#df_agg.reset_index()
#df_agg

In [117]:
# get date range for all dates
all_days = pd.date_range(df_agg.index.get_level_values('date_start').min(), 
                         df_agg.index.get_level_values('date_start').max(), freq='D')


#df_agg.index.get_level_values('date_start')
#df_agg.index.get_level_values('gwno')

#df.reindex(all_days)

# fill with NaNs for missing dates (days) and country codes
# so that we can merge with data from other DBs

#df.loc[all_days]

#repeat for gwno

In [120]:
#all_days = pd.date_range(df_agg.index.min(), df_agg.index.max(), freq='D')
df_agg.index.get_level_values('date_start').max()

'2015-12-31'

In [ ]:
# create empty tabel with desired format 
# join df_agg on empty table